In [12]:
import re
import cobra

In [2]:
model = cobra.io.load_matlab_model('Escherichia_coli_str_K_12_substr_MG1655.mat')

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-29


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


In [14]:
def find_objective_rec(objective_expression_str: str):
    # 正则表达式匹配以 'biomass' 开头的反应ID
    matches = re.findall(r'bio[^\s\*]+', objective_expression_str)
    if matches:
        biomass_reaction = matches[0]
        return biomass_reaction
    

shared_medium = {
    'EX_mn2(e)': 1000.0, 'EX_cu2(e)': 1000.0, 'EX_pi(e)': 1000.0,
    'EX_cobalt2(e)': 1000.0, 'EX_h(e)': 1000.0, 'EX_mg2(e)': 1000.0,
    'EX_co2(e)': 1000.0, 'EX_o2(e)': 1000.0, 'EX_cl(e)': 1000.0,
    'EX_zn2(e)': 1000.0, 'EX_so4(e)': 1000.0, 'EX_fe3(e)': 1000.0,
    'EX_h2o(e)': 1000.0, 'EX_k(e)': 1000.0, 'EX_nh4(e)': 1000.0,
    'EX_ca2(e)': 1000.0, 'EX_na1(e)': 1000.0, 'EX_fe2(e)': 1000.0
}



In [15]:
model = cobra.io.load_matlab_model('Escherichia_coli_str_K_12_substr_MG1655.mat')
biomass_reaction = find_objective_rec(str(model.objective.expression))  # Biomass reaction ID
flux_limit = 1e3  # Set this to the desired maximum flux sum value

            # Add auxiliary variables and constraints to represent absolute fluxes
abs_flux_vars = []
for reaction in model.reactions:
                if reaction.id != biomass_reaction and reaction.id not in shared_medium.keys():
                # Create a new variable to represent the absolute flux
                    abs_flux_var_name = f"abs_flux_{reaction.id}"
                    if abs_flux_var_name not in model.variables:
                        abs_flux_var = model.problem.Variable(f"abs_flux_{reaction.id}", lb=0)
                        abs_flux_vars.append(abs_flux_var)

                    # Add constraints to link the absolute flux variable with the reaction flux
                        model.add_cons_vars([
                            model.problem.Constraint(abs_flux_var - model.reactions.get_by_id(reaction.id).flux_expression, lb=0),
                            model.problem.Constraint(abs_flux_var + model.reactions.get_by_id(reaction.id).flux_expression, lb=0)
                        ])

            # Add the constraint for the sum of absolute fluxes
if "total_abs_flux_constraint" not in model.constraints:
                total_abs_flux_constraint = model.problem.Constraint(sum(abs_flux_vars), ub=flux_limit, name="total_abs_flux_constraint")
                model.add_cons_vars([total_abs_flux_constraint])

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p


In [16]:
from cobra.flux_analysis import pfba
pfba_solution = pfba(model)
growth_rate = pfba_solution.fluxes[find_objective_rec(str(model.objective.expression))]

In [21]:
met_flux = [(rec.id, pfba_solution.fluxes[rec.id]) for rec in model.reactions]
met_flux

[('12PPD_Stex', 0.0),
 ('12PPDtpp', 0.0),
 ('15DAPtex', 0.0),
 ('15DAPtpp', 0.0),
 ('1HIBUPGLU_St2pp', 0.0),
 ('1HIBUPGLU_Stex', 0.0),
 ('1HIBUP_S_GLCAASE', 0.0),
 ('1HIBUP_Stepp', 0.0),
 ('1HIBUP_Stex', 0.0),
 ('1HMDGLUC_GLCAASE', 0.0),
 ('1HMDGLUCt2pp', 0.0),
 ('1HMDGLUCtex', 0.0),
 ('1OHMDZtepp', 0.0),
 ('1OHMDZtex', 0.0),
 ('23DHMPO', 0.0),
 ('23PDE2', 0.0),
 ('23PDE4', 0.0),
 ('23PDE7', 0.0),
 ('23PDE9', 0.0),
 ('2AHBUTI', 0.0),
 ('2DDGLCNtex', 0.0),
 ('2DH3DGLCA', 0.0),
 ('2DMMQ8abcpp', 0.0),
 ('2DMMQ8tex', 0.0),
 ('2HATVACIDGLUC_GLCAASE', 0.0),
 ('2HATVACIDGLUCt2pp', 0.0),
 ('2HATVACIDGLUCtex', 0.0),
 ('2HATVACIDtepp', 0.0),
 ('2HATVACIDtex', 0.0),
 ('2HATVLACGLUC_GLCAASE', 0.0),
 ('2HATVLACGLUCt2pp', 0.0),
 ('2HATVLACGLUCtex', 0.0),
 ('2HATVLACtepp', 0.0),
 ('2HATVLACtex', 0.0),
 ('2HIBUPGLU_St2pp', 0.0),
 ('2HIBUPGLU_Stex', 0.0),
 ('2HIBUP_S_GLCAASE', 0.0),
 ('2HIBUP_Stepp', 0.0),
 ('2HIBUP_Stex', 0.0),
 ('2IMZS', 0.0),
 ('2MBCOATA', 0.0),
 ('2OBUTt2rpp', 0.0),
 ('2OBUTtex', 0